In [142]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [185]:
# Download dataset and prepare data
IRIS_TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"  
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'species'] 
train = pd.read_csv(IRIS_TRAIN_URL, names = names, skiprows= 1)
test = pd.read_csv(IRIS_TEST_URL, names = names, skiprows= 1)
## train and test input data
train_Y = pd.get_dummies(train.species)
test_Y = pd.get_dummies(test.species)

train_X = train.drop("species", axis =1)
test_X = test.drop("species", axis = 1)

In [186]:
def get_next_batch(x, y, start, end):
    batch_x = x[start:end]
    batch_y = y[start:end]
    return batch_x, batch_y

In [187]:
train_X.head()
train_Y.shape

(120, 3)

In [201]:
## set the hyper parameters and network parameters
learning_rate = 0.01
training_epoch = 200
display_step = 200
batch_size = 120

# network parameter  ## one hidden layer
n_input = train_X.shape[1]
n_hidden = 10
n_classes = 3

## graph unit
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

In [202]:
## store weights and biases
weights = {'h1':  tf.Variable(tf.random_normal([n_input, n_hidden])) ,
           'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))}

biases = {'b1': tf.Variable(tf.random_normal([n_hidden])),
          'out': tf.Variable(tf.random_normal([n_classes])) }

In [203]:
## create model
def multilayer_percepton(x):
    ## hidden fully connected layer
    layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['h1']), biases['b1']))
    ## ouput layer
    out_layer = tf.nn.relu(tf.add(tf.matmul(layer_1, weights['out']),biases['out'] ))
    
    return out_layer

In [204]:
## construct model
logits = multilayer_percepton(X)

In [205]:
## define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels= Y ))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate )
train_op = optimizer.minimize(loss_op)

In [206]:
## initialize the variable
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
global_step = 0
## training cycle
for epoch in range(training_epoch):
    avg_cost = 0 
    total_batch = int(train_X.shape[0]/batch_size)
    ## loop over batches
    for iteration in range(total_batch):
        global_step += 1
        start = iteration * batch_size
        end = (iteration + 1) * batch_size
        batch_x, batch_y = get_next_batch(train_X, train_Y, start, end)
        # Run optimization op (backprop) and cost op (to get loss value)
        _, c = sess.run([train_op,loss_op], feed_dict= {X:batch_x, Y:batch_y})
        ## compute avarage loss
        avg_cost+= c/total_batch
        ## display logs per epoch step 
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
print("Optimization Finished!")

## apply softmax to logits
pred = tf.nn.softmax(logits)
correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

## test the network after training

feed_dict_test = {X :test_X, Y:test_Y}
loss_test, acc_test = sess.run([loss_op, accuracy], feed_dict=feed_dict_test)
print('---------------------------------------------------------')
print("Test loss: {0:.2f}, test accuracy: {1:.01%}".format(loss_test, acc_test))
print('---------------------------------------------------------')
sess.close()


## Test loss: 0.09, test accuracy: 93.3% -- epoch 200 learning rate = .01        
        

Epoch: 0001 cost=6.037530899
Optimization Finished!
---------------------------------------------------------
Test loss: 0.53, test accuracy: 93.3%
---------------------------------------------------------
